In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, BatchNormalization, Dropout
import tensorflow as tf

# Set up the ImageDataGenerator for training
# train_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normalize pixel values to [0, 1]
    shear_range=0.2,    # Random shear transformations
    zoom_range=0.2,     # Random zoom
    horizontal_flip=True,  # Allow random horizontal flipping
    validation_split=0.2  # Reserve 20% of data for validation
)

train_generator = train_datagen.flow_from_directory(
    './dataset',  # Update this with the path to your training data
    target_size=(224, 224),   # Image size the model expects
    batch_size=32,
    class_mode='categorical'  # Update this according to your problem ('binary', 'categorical', etc.)
)


Found 1267 images belonging to 4 classes.


In [ ]:

# Define your model (replace with your actual model)
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(train_generator.class_indices), activation='sigmoid')  # Adjust for the number of classes
])



model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Calculate steps_per_epoch: Total number of samples / batch size
steps_per_epoch = train_generator.samples // train_generator.batch_size

# Train the model
history = model.fit(
    train_generator, 
    epochs=20,
    # validation_split=0.2
    steps_per_epoch=steps_per_epoch  # This tells Keras how many batches per epoch

)


C:\Users\71518\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20


C:\Users\71518\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


39/39 ━━━━━━━━━━━━━━━━━━━━ 35s 790ms/step - accuracy: 0.5761 - loss: 1.0206
Epoch 2/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 1.0000 - loss: 5.5088e-04   
Epoch 3/20


c:\Program Files\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


39/39 ━━━━━━━━━━━━━━━━━━━━ 60s 1s/step - accuracy: 1.0000 - loss: 0.0014
Epoch 4/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 120us/step - accuracy: 1.0000 - loss: 3.8854e-06 
Epoch 5/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 38s 850ms/step - accuracy: 1.0000 - loss: 6.3087e-06
Epoch 6/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 0s/step - accuracy: 1.0000 - loss: 9.7815e-06    
Epoch 7/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 30s 723ms/step - accuracy: 1.0000 - loss: 3.6426e-06
Epoch 8/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 185us/step - accuracy: 1.0000 - loss: 1.7323e-06 
Epoch 9/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 29s 678ms/step - accuracy: 1.0000 - loss: 4.7176e-06
Epoch 10/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 151us/step - accuracy: 1.0000 - loss: 1.0497e-05 
Epoch 11/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 30s 707ms/step - accuracy: 1.0000 - loss: 4.6867e-06
Epoch 12/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 166us/step - accuracy: 1.0000 - loss: 2.5182e-06 
Epoch 13/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 31s 707ms/step - accuracy: 1.0000 - loss: 3.4375e-06
Epoch 14/20
3

In [4]:
# Save the trained model
model.save("face_recognition_model.h5")

In [5]:
class_labels = list(train_generator.class_indices.keys())
print("Class Labels:", class_labels)

Class Labels: ['Abbas', 'Abdulaziz', 'Haitham', 'Ziyad']


In [ ]:
import cv2
from keras.models import load_model
import numpy as np

# Load the trained face recognition model
model = load_model('face_recognition_model.h5')

# Define the class names corresponding to the model's output
names = ["Abbas", "Abdulaziz", "Haitham", "Ziyad"]  # Replace with your actual class names

# Initialize the webcam (0 for the default camera)
cap = cv2.VideoCapture(0)

# Load OpenCV's pre-trained Haar Cascade Classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

print("Instructions:")
print("- Press SPACEBAR to capture an image and recognize the face.")
print("- Press 'q' to quit the program.")

while True:
    # Capture frame-by-frame from the webcam
    ret, frame = cap.read()

    if not ret:
        print("Error: Could not access the webcam.")
        break

    # Display the live camera feed
    cv2.imshow('Camera Feed - Press SPACEBAR to Capture', frame)

    # Wait for key press
    key = cv2.waitKey(1) & 0xFF

    # SPACEBAR captures the frame for processing
    if key == ord(' '):
        # Convert the frame to grayscale for better face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)

        if len(faces) == 0:
            print("No faces detected. Try capturing again.")
            continue

        for (x, y, w, h) in faces:
            # Draw a rectangle around the detected face
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # Extract the face region from the frame
            face = frame[y:y + h, x:x + w]

            # Resize the face to the input size expected by the model (224x224)
            face_resized = cv2.resize(face, (224, 224))

            # Normalize the face image (scale pixel values between 0 and 1)
            face_normalized = face_resized / 255.0
            face_normalized = np.expand_dims(face_normalized, axis=0)  # Add batch dimension

            # Predict the class of the face using the model
            predictions = model.predict(face_normalized)
            predicted_class = np.argmax(predictions, axis=1)[0]  # Get the class index

            # Get the corresponding name for the predicted class
            predicted_name = names[predicted_class]

            # Print the predicted name in the console
            print(f"Recognized: {predicted_name}")

            # Overlay the predicted name on the video feed
            cv2.putText(frame, f"Name: {predicted_name}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Display the captured frame with predictions
        cv2.imshow('Recognition Result', frame)

    # Press 'q' to quit the application
    if key == ord('q'):
        break

# Release the webcam and close OpenCV windows
cap.release()
cv2.destroyAllWindows()


In [2]:
import cv2
from keras.models import load_model
import numpy as np

# Load the trained model
model = load_model('face_recognition_model.h5')

# Specify the class names corresponding to your model's output
names = ["Abbas", "abdulaziz", "Haitham", "Ziyad"]  # Replace with your actual class names

# Initialize the webcam (0 for the default camera)
cap = cv2.VideoCapture(0)

# Specify the cascade for face detection (OpenCV's pre-trained Haar cascade)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

print("Press SPACEBAR to capture an image and recognize the face.")
print("Press 'q' to quit the program.")

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    if not ret:
        print("Failed to capture frame. Exiting...")
        break

    # Display the camera feed
    cv2.imshow('Camera - Press SPACEBAR to Capture', frame)

    # Wait for key press
    key = cv2.waitKey(1) & 0xFF

    # If SPACEBAR is pressed, capture the image
    if key == ord(' '):
        # Convert to grayscale for better face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)

        if len(faces) == 0:
            print("No faces detected. Try again.")
            continue

        for (x, y, w, h) in faces:
            # Draw a rectangle around the detected face
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # Extract the face from the frame
            face = frame[y:y + h, x:x + w]

            # Resize the face to match the input size of the trained model
            face_resized = cv2.resize(face, (224, 224))

            # Normalize the image (same preprocessing as used during training)
            face_normalized = face_resized / 255.0
            face_normalized = np.expand_dims(face_normalized, axis=0)  # Add batch dimension

            # Perform prediction using the trained model
            predictions = model.predict(face_normalized)

            # Get the class with the highest probability (index)
            predicted_class = np.argmax(predictions, axis=1)[0]

            # Get the predicted person's name
            predicted_name = names[predicted_class]

            # Display the predicted name 
            print(f"Predicted Name: {predicted_name}")

            # Display the name on the captured image
            cv2.putText(frame, f"Name: {predicted_name}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Show the captured frame with predictions
        cv2.imshow('Captured Image - Prediction', frame)

    # If 'q' is pressed, exit the loop
    if key == ord('q'):
        break

# Release the capture object and close all windows
cap.release()
cv2.destroyAllWindows()


Press SPACEBAR to capture an image and recognize the face.
Press 'q' to quit the program.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
Predicted Name: abdulaziz
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Predicted Name: abdulaziz
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Predicted Name: abdulaziz
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Predicted Name: abdulaziz
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Predicted Name: abdulaziz
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Predicted Name: abdulaziz


In [ ]:
import cv2
from deepface import DeepFace
from keras.models import load_model
import numpy as np

# Load the trained model
model = load_model('face_recognition_model.h5')

# Initialize the webcam (0 for default camera)
cap = cv2.VideoCapture(0)

# Specify the cascade for face detection (OpenCV's pre-trained Haar cascade)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    if not ret:
        break


    # Convert to grayscale for better face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    for (x, y, w, h) in faces:
        # Draw rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Extract the face from the frame
        face = frame[y:y + h, x:x + w]

        # Resize the face to match the input size of the trained model
        face_resized = cv2.resize(face, (224, 224))

        # Normalize the image (same preprocessing as used during training)
        face_normalized = face_resized / 255.0
        face_normalized = np.expand_dims(face_normalized, axis=0)  # Add batch dimension

        # Perform prediction
        predictions = model.predict(face_normalized)

        # Get the class with the highest probability (index)
        predicted_class = np.argmax(predictions, axis=1)[0]

        # Display the predicted person's name (you can map the class index to person names)
        names = ["Abbas", "abdulaziz", "Haitham", "Ziyad"]  # Replace with your actual class names
        predicted_name = names[predicted_class]

        # Display the name on the image
        cv2.putText(frame, f"Name: {predicted_name}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Show the frame with detected faces and predictions
    cv2.imshow('Camera - Face Recognition', frame)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture object and close all windows
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━